In [14]:
from predict_labels import main
import sys

file = './sample_input/test1.xls' 
ce_model = './sample_models/ce.model' 
fe_model = './sample_models/fe.model' 
cl_model = './sample_models/cl.model' 
w2v = '/Users/majid/DIG/data/glove.840B.300d.txt' 
vocab_size = 500000 
infersent_model = '/Users/majid//DIG/research-workspace/table_understanding_v2/libraries/infersent1.pkl' 
infersent_source = '/Users/majid/DIG/research-workspace/table_understanding_v2/libraries/'


res = main(file, ce_model, fe_model, cl_model, w2v, vocab_size, infersent_source, infersent_model)

loading word vectors...
loading word embeddings...
creating dict...
embeddings loaded!


  1%|▏         | 1/77 [00:00<00:07,  9.94it/s]

initialize 77 text sentences...


100%|██████████| 77/77 [00:02<00:00, 26.44it/s]
0it [00:00, ?it/s]


initialize 0 numeric sentences...


In [15]:
import bokeh
from bokeh.models import ColumnDataSource, TableColumn, DateFormatter, DataTable, CustomJS
from bokeh.layouts import column
from bokeh.io import show, curdoc, output_notebook, output_file
from bokeh.models.widgets import TextInput
from bokeh.plotting import curdoc
import pandas as pd
import numpy as np
from ipywidgets import HBox, Layout, Style, Button

output_notebook()


Loading BokehJS ...

In [16]:
tarr = list(res.values())[0]['text']
labels = list(res.values())[0]['labels']
probs = list(res.values())[0]['labels_probs']

df = pd.DataFrame(tarr, columns=[str(i) for i, _ in enumerate(tarr[0])])
data = df.to_dict('series')
source = ColumnDataSource(data=data)
columns = [
    TableColumn(field=str(i), title=str(i)) for i in range(len(df.columns))
]
data_table = DataTable(source = source, columns = columns, width = 1400, height=3000, 
                       editable = False, reorderable = False)

source_code = """
var table = document.getElementsByClassName('grid-canvas')[0];
var rows = document.getElementsByClassName('slick-row');
var count = $('#gvPerformanceResult div').length;

//for (var i=0,max = rows.length ; i<max ; i++){
//    if(rows[i].className.includes("active")){
//        cells = rows[i].getElementsByClassName('slick-cell');
//        for(var j=0,max_c = cells.length ; j<max_c ; j++){
//            if(cells[j].className.includes("active")){
//                console.log("active cell: ", i, j);
//                cells[j].style.backgroundColor = "rgba(0, 0, 0, 0.2)";
//            }
//        }
//    }
//}

for (var i=0,max = rows.length ; i<max ; i++){
    cells = rows[i].getElementsByClassName('slick-cell');
    for(var j=1,max_c = cells.length ; j<max_c ; j++){
        text = cells[j].getElementsByClassName('bk')[0].innerHTML.trim();
        if(text == ""){
            continue;
        }
        cl = classes[i][j-1];
        cells[j].style.backgroundColor = "rgba("+rgb[cl][0]+", "+rgb[cl][1]+", "+rgb[cl][2]+", "+opacity[i][j-1]+")";
    }
}

console.log("majid", count, rows.length)
"""

N, M = df.values.shape
opacity = probs

l2i = ['attributes', 'data', 'header', 'metadata', 'derived', 'notes']
classes = [[l2i.index(l) for l in row] for row in labels]
RGB = [[104,54,154], [66,255,59], [66,117,244], [184,95,41], [255,0,0], [184,173,41]]

layout=Layout(width='60pt', color='blue')
legends = [Button(description=l, layout=layout, style={'button_color':f'rgba({c[0]},{c[1]},{c[2]}, 0.3)'}) for l, c in zip(l2i, RGB)]
display(HBox(legends))

callback = CustomJS(args = dict(source = source, opacity = opacity, classes=classes, rgb=RGB), code = source_code)
source.selected.js_on_change('indices', callback)
source.selected.js_on_event('tap', callback)

curdoc().add_root(column(data_table))

show(data_table)